In [2]:
import numpy as np
import pandas as pd
from pandas import DataFrame
from pandas import Series
import seaborn as sns
from tqdm import tqdm
import plotly.io as pio
import plotly.express as px
import plotly.offline as pyo
import plotly.graph_objs as go
import matplotlib.font_manager as fm
font_path = './AppleGothic.ttf'
fontprop = fm.FontProperties(fname=font_path, size=18)
font_name = fm.FontProperties(fname=font_path).get_name()

In [40]:
# 고객 및 주거래계좌 정보(CUS_INFO.CSV)
cus_info = pd.read_csv('cus_info_new.csv')

# 종목정보(IEM_INFO.CSV)
iem_info = pd.read_csv('종목정보_시장정보처리.csv', encoding='cp949')

# 국내주식 잔고이력(STK_BNC_HIST.CSV) _ 잔고정보 (국내 주식건에 한해) <- 여기세 종목 정보 들어있음
stk_bnc_hist = pd.read_csv('stk_bnc_hist.csv')

# 국내주식 보유기간(STK_HLD_.CSV)
stk_hld_test = pd.read_csv('stk_hld_test.csv')
stk_hld_train = pd.read_csv('stk_hld_train.csv')

In [41]:
cus= cus_info.copy()
iem = iem_info.copy()
hist = stk_bnc_hist.copy()
train = stk_hld_train.copy()
test = stk_hld_test.copy()

In [42]:
cus.columns=["계좌 ID","성별","연령대","투자성향","자산구간","주거래상품군","Life Style","서비스 등급","총 투자기간","주거래업종구분"]
iem.columns= ["종목코드","종목한글명","종목업종", "시가총액 규모유형","시장구분"]
hist.columns= ["계좌 ID","기준일자","종목코드","잔고수량","잔고금액","주당 액면가"]
train.columns=["계좌 ID","종목코드","매수일자","보유기간(일)"]
test.columns=["계좌 ID","종목코드","매수 일자","과거 보유일","제출ID","보유기간(일)"]

In [14]:
aa = test[['계좌 ID','종목코드','과거 보유일']]

In [15]:
aa.drop_duplicates()

,계좌 ID,종목코드,과거 보유일
0,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A032640,153
1,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A160600,335
2,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A234340,139
3,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A131760,236
4,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A293490,9
...,...,...,...
70591,564af3cf975ed4fa05148ada689d5002e9c2d43dee9481...,A000810,23
70592,564af3cf975ed4fa05148ada689d5002e9c2d43dee9481...,A005945,103
70593,4561928f2825f6389f228088ac807a9fb4575ffdcbc47a...,A004740,858
70594,4561928f2825f6389f228088ac807a9fb4575ffdcbc47a...,A001630,11


# hist_d 생성

## 연도별로 자르기

In [3]:
train_eco = pd.read_csv("train_경제지표.csv", encoding='cp949')

In [3]:
train_eco

,계좌 ID,종목코드,매수일자,보유기간(일),국제유가,광물가격,기준금리,환율
0,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A006360,2018-07-26,11,69.14,6266.47,1.50,1134.08
1,f431b2a7651bccfc7ce8f294fdacdf0b7b31db734be701...,A023590,2018-07-26,5,69.14,6266.47,1.50,1134.08
2,43a0eb89f16d3a46767794dda9c31cd15e1ff9daf33332...,A010820,2018-07-26,3,69.14,6266.47,1.50,1134.08
3,5b6973bff9ab839507cf2f839b8b6d190d8ac2144fa5a4...,A005930,2018-07-26,122,69.14,6266.47,1.50,1134.08
4,1c52768eebea2eab6a7ed8cc95d838a3732fc6dce51ba3...,A006360,2018-07-26,2,69.14,6266.47,1.50,1134.08
...,...,...,...,...,...,...,...,...
681467,bcbf85d4382788cc37e75b11ab141e3bf7cde87ff7253d...,A267250,2019-03-27,89,59.73,6339.61,1.75,1133.76
681468,6d78a1da2ff49b7be5a15a22134c743e0ed478f58bb6f5...,A253450,2019-03-27,7,59.73,6339.61,1.75,1133.76
681469,ed62927d381883a19affcc2097336e801e2212ba0cc56d...,A091990,2019-03-27,4,59.73,6339.61,1.75,1133.76
681470,5eafe957081039a4c9e71186cdd1c74dfd674f00be01cc...,A034940,2019-03-27,1,59.73,6339.61,1.75,1133.76


In [4]:
train_eco['year']= train_eco['매수일자'].apply(lambda x: x[0:4])

In [5]:
train_eco

,계좌 ID,종목코드,매수일자,보유기간(일),국제유가,광물가격,기준금리,환율,year
0,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A006360,2018-07-26,11,69.14,6266.47,1.50,1134.08,2018
1,f431b2a7651bccfc7ce8f294fdacdf0b7b31db734be701...,A023590,2018-07-26,5,69.14,6266.47,1.50,1134.08,2018
2,43a0eb89f16d3a46767794dda9c31cd15e1ff9daf33332...,A010820,2018-07-26,3,69.14,6266.47,1.50,1134.08,2018
3,5b6973bff9ab839507cf2f839b8b6d190d8ac2144fa5a4...,A005930,2018-07-26,122,69.14,6266.47,1.50,1134.08,2018
4,1c52768eebea2eab6a7ed8cc95d838a3732fc6dce51ba3...,A006360,2018-07-26,2,69.14,6266.47,1.50,1134.08,2018
...,...,...,...,...,...,...,...,...,...
681467,bcbf85d4382788cc37e75b11ab141e3bf7cde87ff7253d...,A267250,2019-03-27,89,59.73,6339.61,1.75,1133.76,2019
681468,6d78a1da2ff49b7be5a15a22134c743e0ed478f58bb6f5...,A253450,2019-03-27,7,59.73,6339.61,1.75,1133.76,2019
681469,ed62927d381883a19affcc2097336e801e2212ba0cc56d...,A091990,2019-03-27,4,59.73,6339.61,1.75,1133.76,2019
681470,5eafe957081039a4c9e71186cdd1c74dfd674f00be01cc...,A034940,2019-03-27,1,59.73,6339.61,1.75,1133.76,2019


In [5]:
train_eco['과거 보유일'] = np.nan

In [6]:
train_eco2016 = train_eco[train_eco['year']=='2016'].copy()
train_eco2016['과거 보유일'] = train_eco2016['보유기간(일)']*0.5

train_eco2017 = train_eco[train_eco['year']=='2017'].copy()
train_eco2017['과거 보유일'] = train_eco2017['보유기간(일)']*0.6

train_eco2018 = train_eco[train_eco['year']=='2018'].copy()
train_eco2018['과거 보유일'] = train_eco2018['보유기간(일)']*0.7

train_eco2019 = train_eco[train_eco['year']=='2019'].copy()
train_eco2019['과거 보유일'] = train_eco2019['보유기간(일)']*0.8

train_eco2020 = train_eco[train_eco['year']=='2020'].copy()
train_eco2020['과거 보유일'] = train_eco2020['보유기간(일)']*0.9

In [7]:
train_year =pd.concat([train_eco2016,train_eco2017,train_eco2018,train_eco2019,train_eco2020], ignore_index=True)

In [8]:
train_year['과거 보유일'] = round(train_year['과거 보유일'],2)

In [9]:
train_year = train_year.drop('year', axis=1, inplace=False)

In [10]:
iem = pd.read_csv("iem_new2.csv", encoding='cp949')

In [11]:
cus =pd.read_csv("cus_info_new.csv", encoding='cp949')

In [12]:
cus.columns=["계좌 ID","성별","연령대","투자성향","자산구간","주거래상품군","Life Style","서비스 등급","총 투자기간","주거래업종구분"]

In [29]:
train_data= pd.merge(train_year,iem, on='종목코드')
train_data = pd.merge(train_data , cus, on='계좌 ID')

In [14]:
test = pd.read_csv("test_경제지표.csv", encoding='cp949')

In [28]:
test_data= pd.merge(test, iem, on='종목코드')
test_data = pd.merge(test_data , cus, on='계좌 ID')

In [30]:
from sklearn.preprocessing import LabelEncoder
L_encoder = LabelEncoder()
L_encoder.fit(iem["종목한글명"]) # 종목 한글명
train_data["종목한글명"] = L_encoder.transform(train_data["종목한글명"])
test_data["종목한글명"] = L_encoder.transform(test_data["종목한글명"])

In [31]:
test_data.head()

,계좌 ID,종목코드,매수일자,과거 보유일,제출ID,보유기간(일),국제유가,광물가격,기준금리,환율,...,시장구분,성별,연령대,투자성향,자산구간,주거래상품군,Life Style,서비스 등급,총 투자기간,주거래업종구분
0,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A032640,2020-05-22,153,IDX00001,0,33.78,5401.56,0.75,1227.40,...,1,1,9,3,2,2,9,5,5,8
1,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A160600,2019-08-23,335,IDX00002,0,55.44,5697.91,1.50,1205.28,...,4,1,9,3,2,2,9,5,5,8
2,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A234340,2020-06-11,139,IDX00003,0,39.45,5796.61,0.50,1199.39,...,4,1,9,3,2,2,9,5,5,8
3,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A131760,2020-01-20,236,IDX00004,0,58.40,6308.35,1.25,1159.24,...,4,1,9,3,2,2,9,5,5,8
4,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A293490,2020-12-17,9,IDX00005,0,47.75,7833.29,0.50,1092.63,...,2,1,9,3,2,2,9,5,5,8


In [32]:
train_data.head()

,계좌 ID,종목코드,매수일자,보유기간(일),국제유가,광물가격,기준금리,환율,과거 보유일,종목한글명,...,시장구분,성별,연령대,투자성향,자산구간,주거래상품군,Life Style,서비스 등급,총 투자기간,주거래업종구분
0,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A211270,2016-03-28,40,39.58,5043.43,1.5,1165.51,20.0,5,...,4,1,9,3,2,2,9,5,5,8
1,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A099190,2018-05-18,12,71.48,6857.45,1.5,1077.27,8.4,1714,...,4,1,9,3,2,2,9,5,5,8
2,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A099190,2018-03-15,14,60.94,7024.37,1.5,1065.69,9.8,1714,...,4,1,9,3,2,2,9,5,5,8
3,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A014470,2016-03-25,98,40.06,5043.43,1.5,1159.17,49.0,1288,...,4,1,9,3,2,2,9,5,5,8
4,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A028670,2016-01-04,93,37.00,4635.69,1.5,1171.08,46.5,2698,...,1,1,9,3,2,2,9,5,5,8


In [25]:
train_data = train_data.drop(["계좌 ID", "종목코드", "매수일자"], axis = 1) # 계좌번호, 종목코드, 매수일, 기준일자
test_data = test_data.drop(["계좌 ID", "종목코드", "매수일자", "제출ID"], axis = 1)

## automl

In [20]:
import pycaret
pycaret.__version__

'2.3.3'

In [17]:
import pycaret

In [18]:
import pandas as pd
from pandas import Series

In [19]:
from pycaret.regression import *

In [20]:
train_data.head()

,계좌 ID,종목코드,매수일자,보유기간(일),국제유가,광물가격,기준금리,환율,과거 보유일,종목한글명,종목업종,시가총액 규모유형,시장구분,성별,연령대,투자성향,자산구간,주거래상품군,Life Style,서비스 등급,총 투자기간,주거래업종구분
0,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A211270,2016-03-28,40,39.58,5043.43,1.5,1165.51,20.0,5,13,3,4,1,9,3,2,2,9,5,5,8
1,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A099190,2018-05-18,12,71.48,6857.45,1.5,1077.27,8.4,1714,10,2,4,1,9,3,2,2,9,5,5,8
2,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A099190,2018-03-15,14,60.94,7024.37,1.5,1065.69,9.8,1714,10,2,4,1,9,3,2,2,9,5,5,8
3,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A014470,2016-03-25,98,40.06,5043.43,1.5,1159.17,49.0,1288,2,3,4,1,9,3,2,2,9,5,5,8
4,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A028670,2016-01-04,93,37.00,4635.69,1.5,1171.08,46.5,2698,6,2,1,1,9,3,2,2,9,5,5,8


In [21]:
#list of columns that are categorical
cat_f = ['계좌 ID','종목코드','매수일자','종목한글명','종목업종','시가총액 규모유형','시장구분','성별','연령대',
         '투자성향','자산구간','주거래상품군','Life Style','서비스 등급','총 투자기간','주거래업종구분']

In [33]:
exp_reg101 = setup(data = train_data, target = '보유기간(일)', categorical_features = cat_f, ignore_features=["계좌 ID", "종목코드", "매수일자"] )

IntProgress(value=0, description='Processing: ', max=3)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,00:36:54
Status,. . . . . . . . . . . . . . . . . .,Preprocessing Data


Text(value="Following data types have been inferred automatically, if they are correct press enter to continue…

,Data Type
국제유가,Numeric
광물가격,Numeric
기준금리,Numeric
환율,Numeric
과거 보유일,Numeric
종목한글명,Categorical
종목업종,Categorical
시가총액 규모유형,Categorical
시장구분,Categorical
성별,Categorical


MemoryError: Unable to allocate 5.05 GiB for an array with shape (477030, 2839) and data type float32

In [ ]:
compare_models()